In [1]:
from qiime2 import Artifact
import qiime2 as q2
import biom
import pandas as pd

In [2]:
SEQ_TYPE='shotgun'
REP_COL='replicate_id'

In [3]:
#import metadata
mf = q2.Metadata.load('14332_20220503-093430.txt'.format(SEQ_TYPE=SEQ_TYPE)).to_dataframe()
mf = mf.loc[
#     (mf.metagenomic_spike_in != 'TRUE')
    (mf.genomic_spike_in != 'TRUE')
]
mf

,bacteria,bacteria_function,biomass,buffer_removal,col,collection_timestamp,compressed_well,description,diet,dna_conc,...,sample_type,sample_type2,scientific_name,sex,storage_solution,swab_type,taxon_id,title,vivarium,water
sample_name,,,,,,,,,,,,,,,,,,,,,
14332.361158590,not applicable,not applicable,low,dry,10.0,3/25/22,K10,surface_swab_floor,not applicable,0.740,...,surface,surface_swab_floor,indoor metagenome,not applicable,isopropanol,swube,1256227.0,Matrix_pipeline_validation,not applicable,not applicable
14332.361158597,not applicable,not applicable,low,dry,8.0,3/24/22,O8,skin_hand_right,not applicable,0.731,...,skin,skin_hand_right,human skin metagenome,female,isopropanol,swube,539655.0,Matrix_pipeline_validation,not applicable,not applicable
14332.361158598,not applicable,not applicable,low,dry,11.0,3/25/22,P11,surface_swab_keyboard,not applicable,0.743,...,surface,surface_swab_keyboard,indoor metagenome,not applicable,isopropanol,custom,1256227.0,Matrix_pipeline_validation,not applicable,not applicable
14332.361162759,not applicable,not applicable,low,dry,3.0,3/24/22,E3,skin_armpit_right,not applicable,0.736,...,skin,skin_armpit_right,human skin metagenome,male,etoh,swube,539655.0,Matrix_pipeline_validation,not applicable,not applicable
14332.361162761,not applicable,not applicable,low,dry,5.0,3/24/22,G5,skin_armpit_right,not applicable,0.749,...,skin,skin_armpit_right,human skin metagenome,female,etoh,swube,539655.0,Matrix_pipeline_validation,not applicable,not applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14332.CONTROL.7.4.2,not applicable,not applicable,low,dry,12.0,4/4/22,D12,POSITIVE_CONTROL,not applicable,0.735,...,control positive,POSITIVE_CONTROL,metagenome,not applicable,not applicable,not applicable,256318.0,Matrix_pipeline_validation,not applicable,not applicable
14332.CONTROL.STOCK.2.1,not applicable,not applicable,low,dry,2.0,4/4/22,M2,POSITIVE_CONTROL,not applicable,0.841,...,control positive,POSITIVE_CONTROL,metagenome,not applicable,not applicable,not applicable,256318.0,Matrix_pipeline_validation,not applicable,not applicable
14332.CONTROL.STOCK.2.2,not applicable,not applicable,low,dry,3.0,4/4/22,A3,POSITIVE_CONTROL,not applicable,0.841,...,control positive,POSITIVE_CONTROL,metagenome,not applicable,not applicable,not applicable,256318.0,Matrix_pipeline_validation,not applicable,not applicable


In [4]:
# #create a new storage temp condition for samples that were put in -20C after 1 week
mf[REP_COL] = mf.apply(lambda row: row.host_subject_id + ' ' + row.sample_type2, axis=1)
mf.to_csv('{SEQ_TYPE}/metadata.tsv'.format(SEQ_TYPE=SEQ_TYPE), sep='\t')
mf
mf = mf[~(mf.index.isin([
    '14332.363197859', '14332.361164111',
    '14332.363197857', '14332.363197849',
    '14332.363197875', 
    '14332.363197831', '14332.363197893']))]

In [5]:
#import table into qiime2 and filter out blanks, controls, and mistakes

#import table 
bt = biom.load_table('{SEQ_TYPE}/qiita_feature-table.biom'.format(SEQ_TYPE=SEQ_TYPE))
ids = {i: i[7:] for i in bt.ids()}
bt = bt.update_ids(ids, axis='sample', inplace=False)

#identify the metadata data to keep 
keep = [ind for ind in mf.index if 'CONTROL' not in ind and 'BLANK' not in ind]

#indentify the biom table data to keep (match to metadata)
keep2 = list(set(keep) & set(bt.ids()))

#filter metadata
mf_filtered = mf.loc[keep2, : ]

#filter biom table
bt_filter = bt.filter(mf_filtered.index, axis='sample')

#some features may have been removed that were unique to the samples removed
#now there may be a lot of 0's in our table, remove them and match tables
keep_feat = bt_filter.ids('observation')[bt_filter.sum(axis='observation') > 0]
bt_filter = bt_filter.filter(keep_feat, axis='observation')

#turn tables to q2 artifacts
q2mf_filtered = q2.Metadata(mf_filtered)

In [6]:
len(bt_filter.ids())

253

In [8]:
if SEQ_TYPE == 'shotgun':
    tax_to_gotu = {}
    with open('{SEQ_TYPE}/wol-tax.tsv'.format(SEQ_TYPE=SEQ_TYPE)) as f:
        lines = f.readlines()
        for l in lines:
            tax, gotu = l.split('\t')
            gotu = gotu.replace('\n', '').partition(';t__')[0]
            tax_to_gotu[gotu.replace('\n', '')] = tax
    bt_filter.update_ids(tax_to_gotu, axis='observation')
    tax_to_gotu

In [10]:
# q2mf_filtered.save('16s-filtered-metadata.qza')
q2bt = q2.Artifact.import_data('FeatureTable[Frequency]', bt_filter)
q2bt.save('{SEQ_TYPE}/filtered-table.qza'.format(SEQ_TYPE=SEQ_TYPE))

'shotgun/filtered-table.qza'